In [1]:
import os
import joblib
import pandas as pd
import data
import metrics
import app
import numpy as np

In [2]:
def evaluate_model(*, model, metric, X_train, y_train, X_test, y_test,dataset):
    train_predictions = model.predict(X_train)
    test_predictions = model.predict(X_test)
    train_error = metric(y_train, train_predictions)
    test_error = metric(y_test, test_predictions)
    return {
        "train_predictions": train_predictions,
        "test_predictions": test_predictions,
        "train_error": train_error,
        "test_error": test_error
    }

def evaluate_model_baseline(*, model, metric, X_train, y_train, X_test, y_test,dataset):
    train_predictions = model.predict(X_train)
    test_predictions = model.predict(X_test)
    train_error = metric(y_train, train_predictions,X_train)
    test_error = metric(y_test, test_predictions,X_test)
    return {
        "train_predictions": train_predictions,
        "test_predictions": test_predictions,
        "train_error": train_error,
        "test_error": test_error
    }

def print_report(*, model, evaluation):
    print(f"Model used:\n\t{model}")
    print(f"Error:\n\ttrain set {evaluation['train_error']}\n\tTest error: {evaluation['test_error']}")

## Baseline

In [ ]:
dataset = app._get_dataset({'filepath': "./NewDataset/New_dataset.csv"})

In [ ]:
model_path = os.path.join("models", "Baseline", "model.joblib")
model = joblib.load(model_path)
evaluation = evaluate_model_baseline(
    model=model,
    metric=metrics.custom_error,
    X_train=dataset["train"][0],
    y_train=dataset["train"][1],
    X_test=dataset["test"][0],
    y_test=dataset["test"][1],
    dataset = dataset
)
print_report(model=model, evaluation=evaluation)

# Logistic Regression

In [3]:
dataset = app._get_dataset({'filepath': "./NewDataset/New_dataset.csv"})

In [4]:
model_path = os.path.join("models", "2021-11-06 15-18", "model.joblib")
model = joblib.load(model_path)
evaluation = evaluate_model(
    model=model,
    #metric=metrics.mean_absolute_error,
    metric=metrics.custom_error,
    X_train=dataset["train"][0],
    y_train=dataset["train"][1],
    X_test=dataset["test"][0],
    y_test=dataset["test"][1],
    dataset = dataset
)
print_report(model=model, evaluation=evaluation)

Model used:
	Pipeline(steps=[('logistic-regressor', LogisticRegression(random_state=0))])
Error:
	train set {'cnt_error': 363.0, 'total_money': 36401.97574729392}
	Test error: {'cnt_error': 0.0, 'total_money': 0.0}


## gridsearchcv

In [ ]:
x = pd.DataFrame([[1,2,3],[2,3,4],[3,4,5],[4,5,6],[5,6,7],[6,7,8],[7,8,9],[8,9,10],[9,10,11],[10,11,12]])
y = pd.DataFrame([1,2,3,4,5,6,7,8,9,10])

In [ ]:
class BlockingTimeSeriesSplit():
    def __init__(self, n_splits):
        self.n_splits = n_splits
    
    def get_n_splits(self, X, y, groups):
        return self.n_splits
    
    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        k_fold_size = n_samples // self.n_splits
        indices = np.arange(n_samples)

        margin = 0
        for i in range(self.n_splits):
            start = i * k_fold_size
            stop = start + k_fold_size
            mid = int(0.8 * (stop - start)) + start
            yield indices[start: mid], indices[mid + margin: stop]

In [ ]:
indices = np.arange(10)
yield indices[0:2]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
model = LogisticRegression()
btscv = BlockingTimeSeriesSplit(n_splits=5)
scores = cross_val_score(model, x,y, cv=btscv, scoring='r2')
print("Loss: {0:.3f} (+/- {1:.3f})".format(scores.mean(), scores.std()))

In [ ]:
X_train=dataset["train"][0]

In [12]:
df = pd.read_csv('./Datasets/sales_train.csv')
df.head(10)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
5,10.01.2013,0,25,2564,349.00,1.0
6,02.01.2013,0,25,2565,549.00,1.0
7,04.01.2013,0,25,2572,239.00,1.0
8,11.01.2013,0,25,2572,299.00,1.0
9,03.01.2013,0,25,2573,299.00,3.0
